<a href="https://colab.research.google.com/github/Nishanthini03/Custom_Object_Detection/blob/main/Custom_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, shutil
import random

def split_and_copy(source_folder, target_root, class_id):
    files = [f for f in os.listdir(source_folder) if f.endswith(('.png'))]
    random.shuffle(files)
    train_split = int(len(files) * 0.8)
    train_files = files[:train_split]
    val_files = files[train_split:]

    for phase, file_list in zip(['train', 'val'], [train_files, val_files]):
        for f in file_list:
            shutil.copy(os.path.join(source_folder, f), os.path.join(target_root, 'images', phase, f))

os.makedirs('animals_yolo/images/train', exist_ok=True)
os.makedirs('animals_yolo/images/val', exist_ok=True)
os.makedirs('animals_yolo/labels/train', exist_ok=True)
os.makedirs('animals_yolo/labels/val', exist_ok=True)

split_and_copy('/content/drive/MyDrive/AI_Internship/task3/animals/dog', 'animals_yolo', class_id=0)
split_and_copy('/content/drive/MyDrive/AI_Internship/task3/animals/cat', 'animals_yolo', class_id=1)


In [ ]:
!zip -r animals_yolo.zip animals_yolo/

  adding: animals_yolo/ (stored 0%)
  adding: animals_yolo/images/ (stored 0%)
  adding: animals_yolo/images/val/ (stored 0%)
  adding: animals_yolo/images/val/00007-4122619880.png (deflated 0%)
  adding: animals_yolo/images/val/00199-200124509.png (deflated 0%)
  adding: animals_yolo/images/val/00517-3846168679.png (deflated 0%)
  adding: animals_yolo/images/val/00510-3846168672.png (deflated 0%)
  adding: animals_yolo/images/val/00554-3846168716.png (deflated 0%)
  adding: animals_yolo/images/val/00518-3846168680.png (deflated 0%)
  adding: animals_yolo/images/val/00542-3846168704.png (deflated 0%)
  adding: animals_yolo/images/val/00520-3846168682.png (deflated 0%)
  adding: animals_yolo/images/val/00011-4122619884.png (deflated 0%)
  adding: animals_yolo/images/val/51502305736yllmbimoyvsja3byk27ta7f727tfuedsgvjshiqn0eejylk2bqdifjjyk6bg2hbgyizzrb34i7ekjcncgrfq5x5dj7p6xxrj0mer.png (deflated 0%)
  adding: animals_yolo/images/val/00500-3846168662.png (deflated 0%)
  adding: animals_yol

In [ ]:
from google.colab import files
files.download('animals_yolo.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Code for the metric**

In [ ]:
import numpy as np

def compute_iou(box1, box2):
    # box format: [x_center, y_center, width, height]
    x1_min = box1[0] - box1[2] / 2
    y1_min = box1[1] - box1[3] / 2
    x1_max = box1[0] + box1[2] / 2
    y1_max = box1[1] + box1[3] / 2

    x2_min = box2[0] - box2[2] / 2
    y2_min = box2[1] - box2[3] / 2
    x2_max = box2[0] + box2[2] / 2
    y2_max = box2[1] + box2[3] / 2

    inter_xmin = max(x1_min, x2_min)
    inter_ymin = max(y1_min, y2_min)
    inter_xmax = min(x1_max, x2_max)
    inter_ymax = min(y1_max, y2_max)

    inter_area = max(0, inter_xmax - inter_xmin) * max(0, inter_ymax - inter_ymin)

    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)

    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0

def custom_metric(box1, box2, img_width, img_height, alpha=0.5, beta=0.3, gamma=0.2):
    iou = compute_iou(box1, box2)

    center_dist = np.sqrt((box1[0] - box2[0]) ** 2 + (box1[1] - box2[1]) ** 2)
    diag = np.sqrt(img_width ** 2 + img_height ** 2)
    normalized_center_dist = center_dist / diag

    ar1 = box1[2] / box1[3]
    ar2 = box2[2] / box2[3]
    ar_sim = 1 - abs(ar1 - ar2) / max(ar1, ar2)

    return alpha * iou + beta * (1 - normalized_center_dist) + gamma * ar_sim


**Example Usage**

In [ ]:
# YOLO box format: [x_center, y_center, width, height] (normalized between 0 and 1)
box_pred = [0.5, 0.5, 0.4, 0.3]
box_true = [0.52, 0.48, 0.35, 0.32]

score = custom_metric(box_pred, box_true, img_width=640, img_height=640)
print("Custom similarity score:", score)


Custom similarity score: 0.8529420138888889


**YOLOv5 Detection on Validation Set**

In [ ]:
%pwd

'/content'

In [ ]:
%cd /content/drive/MyDrive/AI_Internship/task3

/content/drive/MyDrive/AI_Internship/task3


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17400, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 17400 (delta 58), reused 20 (delta 20), pack-reused 17320 (from 3)
Receiving objects: 100% (17400/17400), 16.26 MiB | 12.46 MiB/s, done.
Resolving deltas: 100% (11921/11921), done.
Updating files: 100% (146/146), done.


In [ ]:
%cd yolov5/

/content/drive/MyDrive/AI_Internship/task3/yolov5


In [ ]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 42.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
yaml_text = """
train: /content/drive/MyDrive/AI_Internship/task3/animals_yolo/train
val: /content/drive/MyDrive/AI_Internship/task3/animals_yolo/val

nc: 2
names: ['cat', 'dog']
"""

with open("/content/drive/MyDrive/AI_Internship/task3/animals_yolo.yaml", "w") as f:
    f.write(yaml_text)


In [ ]:
%pwd

'/content/drive/MyDrive/AI_Internship/task3/yolov5'

In [ ]:
!python train.py \
  --img 640 \
  --batch 8 \
  --epochs 20 \
  --data /content/drive/MyDrive/AI_Internship/task3/animals_yolo.yaml \
  --weights yolov5s.pt \
  --name animals_yolov5s


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-04-17 20:14:13.153297: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744920853.434012   10141 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744920853.503172   10141 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

In [ ]:
!python detect.py \
  --weights /content/drive/MyDrive/AI_Internship/task3/yolov5/runs/train/animals_yolov5s/weights/best.pt \
  --source /content/drive/MyDrive/AI_Internship/task3/animals_yolo/images/val \
  --img 640 \
  --save-txt \
  --save-conf \
  --project /content/drive/MyDrive/AI_Internship/task3/yolov5/runs/detect \
  --name exp \
  --exist-ok


detect: weights=['/content/drive/MyDrive/AI_Internship/task3/yolov5/runs/train/animals_yolov5s/weights/best.pt'], source=/content/drive/MyDrive/AI_Internship/task3/animals_yolo/images/val, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/AI_Internship/task3/yolov5/runs/detect, name=exp, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-414-g78daef4b Python-3.11.12 torch-2.6.0+cu124 CPU

Traceback (most recent call last):
  File "/content/drive/MyDrive/AI_Internship/task3/yolov5/detect.py", line 438, in <module>
    main(opt)
  File "/content/drive/MyDrive/AI_Internship/task3/yolov5/detect.py", line 433, in main
    run(**vars(opt))
  File "/usr/local/lib

**Custom Metric Evaluation**

In [ ]:
import os
import numpy as np
from tqdm import tqdm

from custom_metric import custom_metric  # reuse the function we wrote earlier

def read_yolo_labels(label_path):
    with open(label_path, 'r') as f:
        lines = f.readlines()
    return [list(map(float, line.strip().split())) for line in lines]

def evaluate_custom_metric(pred_dir, gt_dir, img_width, img_height):
    total_score = 0
    count = 0

    for fname in tqdm(os.listdir(gt_dir)):
        if not fname.endswith('.txt'):
            continue

        gt_boxes = read_yolo_labels(os.path.join(gt_dir, fname))
        pred_file = os.path.join(pred_dir, fname)

        if not os.path.exists(pred_file):
            continue

        pred_boxes = read_yolo_labels(pred_file)

        for gt in gt_boxes:
            best_score = 0
            for pred in pred_boxes:
                if int(gt[0]) != int(pred[0]):
                    continue  # class mismatch
                score = custom_metric(gt[1:], pred[1:], img_width, img_height)
                best_score = max(best_score, score)
            total_score += best_score
            count += 1

    return total_score / count if count > 0 else 0

# Example usage
if __name__ == "__main__":
    pred_folder = "runs/detect/exp/labels"       # replace with your folder
    gt_folder = "animals_yolo/labels/val"
    img_width = 640
    img_height = 640

    avg_score = evaluate_custom_metric(pred_folder, gt_folder, img_width, img_height)
    print(f"\n🔍 Average Custom Metric Score: {avg_score:.4f}")
